In [1]:
load('../sim_data/independent_Elnet.RData')

In [2]:
dir = '../sim_data'
dim.list = list()
size = c(50, 100, 500, 1000)
idx = 0
for (P in size){
    for (N in size){
        idx = idx + 1
        dim.list[[idx]] = c(P=P, N=N)
    }
}

files = NULL
for (dim in dim.list){
    p = dim[1]
    n = dim[2]
    files = cbind(files, paste0(dir, '/sim_independent_', paste('P', p, 'N', n, sep='_'), '.RData'))
}

In [4]:
avg_FDR = NULL
for (i in 1:length(files)){
    sim_file = files[i]
    load(sim_file, dat <- new.env())
    sub = dat$sim_array[[i]]
    p = sub$p # take true values from 1st replicate of each simulated data
    coef = sub$beta
    coef.true = which(coef != 0)
    
    tt = results_ind_elnet[[i]]$Stab.table
    
    FDR = NULL
    for (r in 1:nrow(tt)){
        FDR = c(FDR, length(setdiff(which(tt[r, ] !=0), coef.true))/sum(tt[r, ]))

    }
    
    avg_FDR = c(avg_FDR, mean(FDR, na.rm=T))
}

In [5]:
table_ind = NULL
tmp_num_select = rep(0, length(results_ind_elnet))
for (i in 1:length(results_ind_elnet)){
    table_ind = rbind(table_ind, results_ind_elnet[[i]][c('n', 'p', 'rou', 'FP', 'FN', 'MSE', 'Stab')])
    tmp_num_select[i] = mean(rowSums(results_ind_elnet[[i]]$Stab.table))
    
}
table_ind = as.data.frame(table_ind)
table_ind$num_select = tmp_num_select
table_ind$FDR = round(avg_FDR,2)

In [6]:
head(table_ind)

n,p,rou,FP,FN,MSE,Stab,num_select,FDR
50,50,0,14.99 ( 0.59 ),0 ( 0 ),0.53 ( 0.03 ),0.2,19.99,0.67
100,50,0,13.46 ( 0.55 ),0 ( 0 ),0.34 ( 0.01 ),0.23,18.46,0.65
500,50,0,13.65 ( 0.56 ),0 ( 0 ),0.26 ( 0 ),0.23,18.65,0.64
1000,50,0,11.76 ( 0.53 ),0 ( 0 ),0.25 ( 0 ),0.27,16.76,0.60
50,100,0,19.45 ( 0.66 ),0.04 ( 0.02 ),0.74 ( 0.06 ),0.19,24.41,0.74
100,100,0,19.2 ( 1.02 ),0 ( 0 ),0.38 ( 0.01 ),0.2,24.20,0.71


In [7]:
# export result
result.table_ind <- apply(table_ind,2,as.character)
rownames(result.table_ind) = rownames(table_ind)
result.table_ind = as.data.frame(result.table_ind)

# extract numbers only for 'n' & 'p'
result.table_ind$n = tidyr::extract_numeric(result.table_ind$n)
result.table_ind$p = tidyr::extract_numeric(result.table_ind$p)
result.table_ind$ratio = result.table_ind$p / result.table_ind$n

result.table_ind = result.table_ind[c('n', 'p', 'ratio', 'Stab', 'MSE', 'FP', 'FN', 'num_select', 'FDR')]
colnames(result.table_ind)[1:3] = c('N', 'P', 'Ratio')

extract_numeric() is deprecated: please use readr::parse_number() instead
extract_numeric() is deprecated: please use readr::parse_number() instead


In [8]:
# convert interested measurements to be numeric
result.table_ind$Stab = as.numeric(as.character(result.table_ind$Stab))
result.table_ind$MSE_mean = as.numeric(substr(result.table_ind$MSE, start=1, stop=4))
result.table_ind$FP_mean = as.numeric(substr(result.table_ind$FP, start=1, stop=4))
result.table_ind$FN_mean = as.numeric(substr(result.table_ind$FN, start=1, stop=4))
result.table_ind$FN_mean[is.na(result.table_ind$FN_mean)] = 0
result.table_ind$num_select = as.numeric(as.character(result.table_ind$num_select))

Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”

In [9]:
result.table_ind

## export
write.table(result.table_ind, '../results_summary/sim_ind_elnet.txt', sep='\t', row.names=F)

N,P,Ratio,Stab,MSE,FP,FN,num_select,FDR,MSE_mean,FP_mean,FN_mean
50,50,1.00,0.20,0.53 ( 0.03 ),14.99 ( 0.59 ),0 ( 0 ),19.99,0.67,0.53,14.9,0.00
100,50,0.50,0.23,0.34 ( 0.01 ),13.46 ( 0.55 ),0 ( 0 ),18.46,0.65,0.34,13.4,0.00
500,50,0.10,0.23,0.26 ( 0 ),13.65 ( 0.56 ),0 ( 0 ),18.65,0.64,0.26,13.6,0.00
1000,50,0.05,0.27,0.25 ( 0 ),11.76 ( 0.53 ),0 ( 0 ),16.76,0.6,0.25,11.7,0.00
50,100,2.00,0.19,0.74 ( 0.06 ),19.45 ( 0.66 ),0.04 ( 0.02 ),24.41,0.74,0.74,19.4,0.04
100,100,1.00,0.20,0.38 ( 0.01 ),19.2 ( 1.02 ),0 ( 0 ),24.20,0.71,0.38,19.2,0.00
500,100,0.20,0.23,0.27 ( 0 ),16.28 ( 0.78 ),0 ( 0 ),21.28,0.67,0.27,16.2,0.00
1000,100,0.10,0.23,0.26 ( 0 ),17.05 ( 0.76 ),0 ( 0 ),22.05,0.69,0.26,17.0,0.00
50,500,10.00,0.13,1.65 ( 0.13 ),30.38 ( 1.24 ),0.58 ( 0.08 ),34.80,0.83,1.65,30.3,0.58
100,500,5.00,0.15,0.5 ( 0.02 ),32.18 ( 1.44 ),0 ( 0 ),37.18,0.81,0.50,32.1,0.00


In [6]:
library(ggplot2)
library(gridExtra)
pdf('../figures_sim/figure_independent_elnet.pdf', height=5, width=7)
result.table_ind$N = as.factor(result.table_ind$N)
fig_ind_stab = ggplot(result.table_ind, aes(x=P, y=Stab, color=N)) + geom_point() + ylab('Stability')
fig_ind_mse = ggplot(result.table_ind, aes(x=P, y=MSE_mean, color=N)) + geom_point() + ylab('MSE')
fig_ind_fp = ggplot(result.table_ind, aes(x=P, y=FP_mean, color=N)) + geom_point() + ylab('False Positives')
fig_ind_fn = ggplot(result.table_ind, aes(x=P, y=FN_mean, color=N)) + geom_point() + ylab('False Negatives')
grid.arrange(fig_ind_stab, fig_ind_mse, fig_ind_fp, fig_ind_fn, ncol=2, top='Independent_ElasticNet')
dev.off()

Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang


pdf 
  2